In [1]:
import yaml
import numpy as np

### Loading an experiment file

In [17]:
def read_experiments(filename, verbose=True):
    with open(r''+filename) as file:
        experiments = yaml.full_load(file)
        if verbose :
            for exp, content in experiments.items():
                print(key, ":", content)
        return experiments

In [8]:
exp = read_experiments('exp.yaml', False)

In [9]:
exp

{'Experience1': {'nb_steps': 1000,
  'nb_sync': 100,
  'tasks': 3,
  'Type1': {'number': 3,
   'type': 'RegretMatching',
   'spec': {'intertia': 0.1, 'fadding': 0.2}},
  'Type2': {'number': 2, 'type': 'FictiousPlay'},
  'Type3': {'number': 3, 'type': 'Random'}},
 'Experience2': {'tasks': 3,
  'Type1': {'number': 3,
   'type': 'RegretMatching',
   'spec': {'intertia': 0.1, 'fadding': 0.2}},
  'Type2': {'number': 2, 'type': 'FictiousPlay'},
  'Type3': {'number': 1, 'type': 'Random'}}}

### Generating experiments (all combinations)

In [3]:
# Available Agent Types :
types = {0 : "RegretMatching", 1 : "FictitiousPlay", 2 : "Random", 3 : "BestResponse", 
         4 : "SpacialAdaptative"}

In [4]:
nb_vehicles = 3 # for 3 vehicles and 4 tasks, 125 possible combinations
nb_tasks = 4

In [5]:
alloc_dim = np.zeros([len(types)]*nb_vehicles)

In [6]:
alloc_dim.shape

(5, 5, 5)

In [7]:
allAlloc = [x for x,_ in np.ndenumerate(alloc_dim)]

In [8]:
len(allAlloc)

125

In [41]:
def create_experiments(nb_steps, nb_agents, nb_tasks, nb_sync=100):
    # Available Types :
    types = {0 : "RegretMatching", 1 : "FictitiousPlay", 2 : "Random", 3 : "BestResponse", 
         4 : "SpatialAdaptative"}
    alloc_dim = np.zeros([len(types)]*nb_vehicles)
    allAlloc = [x for x,_ in np.ndenumerate(alloc_dim)]
    repartitions = []
    exp = {}
    expId = 1
    for a in allAlloc:
        seen = []
        exp_dict = {}
        exp_dict['nb_sync'] = int(nb_sync)
        exp_dict['tasks'] = int(nb_tasks)
        typeId = 1
        repartition = np.zeros(len(types))
        for t in a :
            type_dict={}
            str_type = types[t]
            #print(seen)
            if str_type not in seen :
                seen.append(str_type)
                type_dict["type"] = str_type
                cnt = int((np.array(a)==t).sum())
                type_dict["number"] = cnt
                repartition[t] = cnt
                if str_type == "RegretMatching":
                    # default values
                    dict_spec = {"inertia": 0.1, "fadding" : 0.2}
                    type_dict["spec"] = dict_spec
                # save the agent params
                exp_dict["Type"+str(typeId)] = type_dict
                typeId += 1
        already_done = np.sum([1 if np.all(repartition == r) else 0 for r in repartitions])
        if already_done < 1 :
            # save the experiment setup
            exp["Experience"+str(expId)] = exp_dict
            expId += 1
            # remember done repartitions
            repartitions.append(repartition)
            #print(repartitions)
    return exp

In [42]:
exp_outp = create_experiments(100, 3, 4, nb_sync=100)

In [43]:
exp_outp

{'Experience1': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 3,
   'spec': {'inertia': 0.1, 'fadding': 0.2}}},
 'Experience2': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'FictitiousPlay', 'number': 1}},
 'Experience3': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'Random', 'number': 1}},
 'Experience4': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'BestResponse', 'number': 1}},
 'Experience5': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'SpatialAdaptative', 'number': 1}},
 'Experience6': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {

### Saving an experiment file

In [44]:
def write_exp_file(filename, exp):
    with open(r''+str(filename), 'w') as file:
        yaml.dump(exp, file)

In [45]:
write_exp_file("exp_3vehicles_4tasks.yaml", exp_outp)